In [2]:
import napari
viewer = napari.Viewer()

In [1]:
import numpy as np
import tifffile
import glob
import skimage.util as util
import cv2
import dask
import pandas as pd

import plotly.express as px
import skimage as ski
import glob
import plotly.graph_objects as go

In [2]:
fnames = glob.glob('new_data/train/images/*.tif')

In [3]:
fnames

['new_data/train/images\\01_02_02.tif',
 'new_data/train/images\\01_10_07.tif',
 'new_data/train/images\\01_19_03.tif',
 'new_data/train/images\\01_38_02.tif',
 'new_data/train/images\\01_45_03.tif',
 'new_data/train/images\\02_04_03.tif',
 'new_data/train/images\\02_15_06.tif']

# Convert .npy to .txt

In [4]:
# Loop over all files
for fname in fnames:
    output_img_path = 'new_data/train/images/' + fname.split('\\')[-1]
    output_labels_path = 'new_data/train/labels/' + fname.split('\\')[-1].replace('.tif', '.txt')

    img = ski.io.imread(fname.replace('\\','/'))
    height, width = img.shape[0:2]

    with open(output_labels_path, 'w') as f:
        rect_fnames = glob.glob(fname.replace('.tif', '*.npy'))
        for rect_fname in rect_fnames:
            assigned_class = rect_fname.split('\\')[-1].split('T')[1][0]
            rects = np.load(rect_fname, allow_pickle=True)
            
            for rect in rects:
                min_row, min_col = np.min(rect, axis=0)
                max_row, max_col = np.max(rect, axis=0)

                # # Calculate YOLO format values
                x_center = (min_col + max_col) / (2 * width)
                y_center = (min_row + max_row) / (2 * height)
                box_width = (max_col - min_col) / width
                box_height = (max_row - min_row) / height
                
                # Write to file (class is always 0 as mentioned)
                f.write(f"{assigned_class} {x_center} {y_center} {box_width} {box_height}\n")
        cv2.imwrite(output_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        f.close()